In [1]:
import os
import requests
import json
import boto3
import dotenv

In [2]:
dotenv.load_dotenv("./../.env")

True

### Test health check

In [3]:
url = "http://localhost:9000/2015-03-31/functions/function/invocations"

payload = {
    "resource": "/healthcheck",
    "path": "/healthcheck",
    "httpMethod": "GET",
    "requestContext": {
    },
    "isBase64Encoded": False
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, data=json.dumps(payload), headers=headers)

In [4]:
print(response.json())

{'statusCode': 200, 'headers': {'content-length': '95', 'content-type': 'application/json'}, 'multiValueHeaders': {}, 'body': '{"message":"The server is up since 2023-08-12 12:13:09","start_uct_time":"2023-08-12 12:13:09"}', 'isBase64Encoded': False}


```sh
# Same as 
curl -XPOST "http://localhost:9000/2015-03-31/functions/function/invocations" -d '{
    "resource": "/healthcheck",
    "path": "/healthcheck",
    "httpMethod": "GET",
    "requestContext": {
    },
    "isBase64Encoded": false
}'
```


### Test Recommend Endpoint 

In [5]:
event = {'movie_ids': [1, 2, 3, 4],
         'user_age': 23,
         "sex": "M",
         "topk": 5}

In [6]:
payload = {
    "resource": "/recommend",
    "path": "/recommend",
    "httpMethod": "POST",
    "requestContext": {
        "resourcePath": "/recommend",
        "httpMethod": "POST"
    },
    "body": json.dumps(event),
    "isBase64Encoded": False
}

headers = {
    "Content-Type": "application/json"
}

In [7]:
response = requests.post(url, data=json.dumps(payload), headers=headers)

In [8]:
print(response.json())

{'statusCode': 200, 'headers': {'content-length': '878', 'content-type': 'application/json'}, 'multiValueHeaders': {}, 'body': '[{"movie_id":50,"title":"Usual Suspects, The (1995)","genres":["Crime","Thriller"],"release_year":1995,"origin_title":"Usual Suspects, The","rating":5.0},{"movie_id":858,"title":"Godfather, The (1972)","genres":["Action","Crime","Drama"],"release_year":1972,"origin_title":"Godfather, The","rating":5.0},{"movie_id":912,"title":"Casablanca (1942)","genres":["Drama","Romance","War"],"release_year":1942,"origin_title":"Casablanca","rating":4.999999523162842},{"movie_id":527,"title":"Schindler\'s List (1993)","genres":["Drama","War"],"release_year":1993,"origin_title":"Schindler\'s List","rating":4.999999046325684},{"movie_id":750,"title":"Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)","genres":["Sci-Fi","War"],"release_year":1963,"origin_title":"Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb","rating":4.999999046

```sh
# Same AS
curl -XPOST "http://localhost:9000/2015-03-31/functions/function/invocations" -d '{
    "resource": "/recommend",
    "path": "/recommend",
    "httpMethod": "POST",
    "requestContext": {
        "resourcePath": "/recommend",
        "httpMethod": "POST"
    },
    "body": "{\"movie_ids\": [1, 2, 3, 4], \"user_age\": 23, \"sex\": \"M\", \"topk\": 1}",
    "isBase64Encoded": false
}'
```

In [9]:
REGION = os.getenv("AWS_DEFAULT_REGION", "ap-southeast-1")

In [10]:
session = boto3.Session(region_name=REGION)

In [11]:
lambda_client = session.client('lambda')

In [12]:
with open("./../integration_test/test_healthcheck_payload.json", mode='r', encoding='utf-8') as f:
    payload = f.read()

response = lambda_client.invoke(
    FunctionName='movielens1m-recommender-lambda',
    Payload=payload
)

In [13]:
with open("./../integration_test/test_recommend_payload.json", mode='r', encoding='utf-8') as f:
    payload = f.read()

response = lambda_client.invoke(
    FunctionName='movielens1m-recommender-lambda',
    Payload=payload
)